# Import library

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
FILE_PATH = "../data/"
FILE_NAME = FILE_PATH + "state_NY-CA.csv"

# Load dataset and first looking

In [3]:
df = pd.read_csv(FILE_NAME)

# Display basic information about the dataset
print("Dataset Overview:")
print(df.info())

/tmp/ipykernel_98361/1955382456.py:1: DtypeWarning: Columns (5,22,23,24,26,27,28,29,30,31,32,33,38,43,44,46,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(FILE_NAME)


Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337463 entries, 0 to 1337462
Data columns (total 99 columns):
 #   Column                                    Non-Null Count    Dtype  
---  ------                                    --------------    -----  
 0   activity_year                             1337463 non-null  int64  
 1   lei                                       1337463 non-null  object 
 2   derived_msa-md                            1337463 non-null  int64  
 3   state_code                                1337463 non-null  object 
 4   county_code                               1327993 non-null  float64
 5   census_tract                              1324599 non-null  object 
 6   conforming_loan_limit                     1328209 non-null  object 
 7   derived_loan_product_type                 1337463 non-null  object 
 8   derived_dwelling_category                 1337463 non-null  object 
 9   derived_ethnicity                         1337463 non-null  o

In [4]:
df.head()

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2023,549300JOT0D4J0SZIK67,40140,CA,6071.0,6071010415.0,C,FSA/RHS:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,5540,49.51,94500,62.21,965,2551,44
1,2023,549300JOT0D4J0SZIK67,40900,CA,6017.0,6017031000.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,6314,26.27,113900,108.93,1738,2460,38
2,2023,549300JOT0D4J0SZIK67,40140,CA,6071.0,6071009120.0,C,FHA:First Lien,Single Family (1-4 Units):Manufactured,Ethnicity Not Available,...,NaN,NaN,NaN,4433,43.85,94500,69.90,854,1552,32
3,2023,549300JOT0D4J0SZIK67,10580,NY,36001.0,36001014201.0,C,VA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,5959,15.66,113300,143.60,1516,2219,51
4,2023,549300JOT0D4J0SZIK67,23420,CA,6019.0,6019006405.0,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,5185,28.95,79400,128.00,1606,2196,31


In [5]:
# List column names
print("Columns in the Dataset:")
print(df.columns.tolist())

Columns in the Dataset:
['activity_year', 'lei', 'derived_msa-md', 'state_code', 'county_code', 'census_tract', 'conforming_loan_limit', 'derived_loan_product_type', 'derived_dwelling_category', 'derived_ethnicity', 'derived_race', 'derived_sex', 'action_taken', 'purchaser_type', 'preapproval', 'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage', 'open-end_line_of_credit', 'business_or_commercial_purpose', 'loan_amount', 'loan_to_value_ratio', 'interest_rate', 'rate_spread', 'hoepa_status', 'total_loan_costs', 'total_points_and_fees', 'origination_charges', 'discount_points', 'lender_credits', 'loan_term', 'prepayment_penalty_term', 'intro_rate_period', 'negative_amortization', 'interest_only_payment', 'balloon_payment', 'other_nonamortizing_features', 'property_value', 'construction_method', 'occupancy_type', 'manufactured_home_secured_property_type', 'manufactured_home_land_property_interest', 'total_units', 'multifamily_affordable_units', 'income', 'debt_to_income_ratio',

## Chunking Approval application

### Cleaning data

In [6]:
# Mapping dictionary
action_taken_mapping = {
    1: 'Loan originated',
    2: 'Application approved but not accepted',
    3: 'Application denied',
    4: 'Application withdrawn by applicant',
    5: 'File closed for incompleteness',
    6: 'Purchased loan',
    7: 'Preapproval request denied',
    8: 'Preapproval request approved but not accepted'
}

loan_type_mapping = {
    1: 'Conventional (not insured or guaranteed by FHA, VA, RHS, or FSA)',
    2: 'Federal Housing Administration insured (FHA)',
    3: 'Veterans Affairs guaranteed (VA)',
    4: 'USDA Rural Housing Service or Farm Service Agency guaranteed (RHS or FSA)'
}

loan_purpose_mapping = {
    1: 'Home purchase',
    2: 'Home improvement',
    31: 'Refinancing',
    32: 'Cash-out refinancing',
    4: 'Other purpose',
    5: 'Not applicable'
}


key_identify = ["income", "debt_to_income_ratio",
                "loan_type", "loan_purpose", "loan_amount", "loan_term",
                "property_value", "loan_to_value_ratio",
                "action_taken_description"
                ]

In [7]:
df['action_taken_description'] = df['action_taken'].map(action_taken_mapping)
df['loan_type']                = df['loan_type'].map(loan_type_mapping)
df['loan_purpose']             = df['loan_purpose'].map(loan_purpose_mapping)

df_org = df[key_identify]
df_org = df_org.dropna()

In [8]:
approved_categories = [
    'Loan originated',
    'Application approved but not accepted',
    'Preapproval request approved but not accepted'
]

In [9]:
approved_applications        = df_org[df_org['action_taken_description'].isin(approved_categories)]
loan_originated_applications = df_org[df_org['action_taken_description'].isin(['Loan originated'])]

In [10]:
loan_originated_applications.head()

,income,debt_to_income_ratio,loan_type,loan_purpose,loan_amount,loan_term,property_value,loan_to_value_ratio,action_taken_description
771,104.0,37,Federal Housing Administration insured (FHA),Home purchase,395000.0,360,405000.0,96.5,Loan originated
772,183.0,44,Conventional (not insured or guaranteed by FHA...,Home purchase,505000.0,360,625000.0,80.0,Loan originated
774,79.0,50%-60%,Federal Housing Administration insured (FHA),Home purchase,255000.0,360,265000.0,96.5,Loan originated
775,94.0,45,Conventional (not insured or guaranteed by FHA...,Home purchase,385000.0,360,425000.0,93.0,Loan originated
776,138.0,38,Conventional (not insured or guaranteed by FHA...,Home purchase,415000.0,360,475000.0,88.0,Loan originated


## Convert rows into Documents for LlamaIndex

In [11]:
import os
from dotenv import load_dotenv
dotenv_path = ".env"
load_dotenv(dotenv_path=dotenv_path)

True

In [13]:
from langchain.schema import Document

# Function to convert a dataframe row into a LangChain Document
def row_to_document(row):
    """
    Convert a dataframe row into a LLama Document object for Pinecone upload.
    """
    # Combine relevant fields into a single content block
    content = (
        f"Loan Application: \n"
        f"Income: ${row['income']}, Debt-to-Income Ratio: {row['debt_to_income_ratio']}, "
        f"Loan Type: {row['loan_type']}, Loan Purpose: {row['loan_purpose']}, "
        f"Loan Amount: ${row['loan_amount']}, Loan Term: {row['loan_term']} months, "
        f"Property Value: ${row['property_value']}, Loan-to-Value Ratio: {row['loan_to_value_ratio']}%, "
        f"Action Taken: {row['action_taken_description']}."
    )

    summary = (
        "\nSummary:\n"
        f"The applicant has an income of ${row['income']} and is applying for a {row['loan_purpose']} loan of ${row['loan_amount']}.\n"
        f"The loan term is {row['loan_term']} months, and the property value is ${row['property_value']}, resulting in a loan-to-value ratio of {row['loan_to_value_ratio']}%.\n"
        f"The action taken on this application was: {row['action_taken_description']}.\n"
    )
    
    final_text = content + summary

    # Add metadata for filtering and additional context
    metadata = {
        "income": row["income"],
        "debt_to_income_ratio": row["debt_to_income_ratio"],
        "loan_type": row["loan_type"],
        "loan_purpose": row["loan_purpose"],
        "loan_amount": row["loan_amount"],
        "loan_term": row["loan_term"],
        "property_value": row["property_value"],
        "loan_to_value_ratio": row["loan_to_value_ratio"],
        "action_taken_description": row["action_taken_description"],
    }

    return Document(page_content=final_text, metadata=metadata)

# Convert all rows from the DataFrame into LangChain Documents
documents = [row_to_document(row) for _, row in loan_originated_applications.iterrows()]

# Example of a single Document
print("Example Document:")
print(documents[0])


Example Document:
page_content='Loan Application: 
Income: $104.0, Debt-to-Income Ratio: 37, Loan Type: Federal Housing Administration insured (FHA), Loan Purpose: Home purchase, Loan Amount: $395000.0, Loan Term: 360 months, Property Value: $405000.0, Loan-to-Value Ratio: 96.5%, Action Taken: Loan originated.
Summary:
The applicant has an income of $104.0 and is applying for a Home purchase loan of $395000.0.
The loan term is 360 months, and the property value is $405000.0, resulting in a loan-to-value ratio of 96.5%.
The action taken on this application was: Loan originated.
' metadata={'income': 104.0, 'debt_to_income_ratio': '37', 'loan_type': 'Federal Housing Administration insured (FHA)', 'loan_purpose': 'Home purchase', 'loan_amount': 395000.0, 'loan_term': 360, 'property_value': 405000.0, 'loan_to_value_ratio': 96.5, 'action_taken_description': 'Loan originated'}


In [14]:
for doc in documents[0:5]:
    print(doc.page_content)

Loan Application: 
Income: $104.0, Debt-to-Income Ratio: 37, Loan Type: Federal Housing Administration insured (FHA), Loan Purpose: Home purchase, Loan Amount: $395000.0, Loan Term: 360 months, Property Value: $405000.0, Loan-to-Value Ratio: 96.5%, Action Taken: Loan originated.
Summary:
The applicant has an income of $104.0 and is applying for a Home purchase loan of $395000.0.
The loan term is 360 months, and the property value is $405000.0, resulting in a loan-to-value ratio of 96.5%.
The action taken on this application was: Loan originated.

Loan Application: 
Income: $183.0, Debt-to-Income Ratio: 44, Loan Type: Conventional (not insured or guaranteed by FHA, VA, RHS, or FSA), Loan Purpose: Home purchase, Loan Amount: $505000.0, Loan Term: 360 months, Property Value: $625000.0, Loan-to-Value Ratio: 80.0%, Action Taken: Loan originated.
Summary:
The applicant has an income of $183.0 and is applying for a Home purchase loan of $505000.0.
The loan term is 360 months, and the property

## Upload docs to vector database PineCone

In [15]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", default=None)

In [16]:
from pinecone import Pinecone, Index, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


model_embeddings = ["text-embedding-ada-002"]

PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=PINECONE_API_KEY)
Pinecone_index_name = "loan-application-records"  # Use lowercase letters and hyphens only
if Pinecone_index_name not in pc.list_indexes().names():
    pc.create_index(
        Pinecone_index_name,
        dimension=1536,
        metric="euclidean",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

pc_index = pc.Index(Pinecone_index_name)

# add docs to vectordb
embeddings = OpenAIEmbeddings(model=model_embeddings[0])

namespace = "loan_app_recommandation"

# is_delete = False
# if (is_delete):
#     pc_index.delete(namespace=namespace, delete_all=True)

PineconeVectorStore.from_documents(
        documents,
        index_name=Pinecone_index_name,
        embedding=embeddings,
        namespace=namespace
    )

print("Successfully uploaded docs to Pinecone vector store")


/home/quochungtran/Desktop/ML_project/LLM_project/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
